# Optimized Meal Planning

## Define the model

Data

$$
\begin{aligned}
T, \text{the set of discrete time periods} \\
I, \text{the set of ingredients that are available} \\
N, \text{the set of nutrients that are tracked} \\
R, \text{the set of known recipes} \\
F_{ij}, \text{the amount of nutrient } j \text{ in a unit amount of ingredient } i,\ \forall i \in I,\ j \in N \\
S_{j}, \text{the amount of nutrient } j \text{ that is needed each week for a healthy diet} \\
C_{i}, \text{the unit cost of ingredient } i,\ \forall i \in I \\
V_{i}, \text{the unit volume of ingredient } i,\ \forall i \in I \\
Fr, \text{the max volume of the fridge} \\
Req_{il}, \text{the amount of ingredient i needed for recipe} \ l \ \forall i \in I, l \in R \\
MAX, \text{the maximum number of recipes that can be made in a week} (tentative)
\end{aligned}
$$


Decision variables:
- $x_{ik}$, the amount of ingredient i bought in week k, $\forall i \in I, k \in T$
- $y_{lk}$, the amount of recipe $l$ to make in week k, $\forall l \in R, k \in T$
- $z_{ik}$, the amount of ingredient i to store in the fridge in week k, $\forall i \in I, k \in T$

Linear Program:
\begin{align}
\min \sum_{k \in T} \sum_{i \in I} x_{ik} * C_i \\
\text{s.t. } \sum_{l \in R} \sum_{i \in I} y_{lk} * Req_{il} * F_{ij} \geq S_{j}, \forall k \in T, j \in N \\
\sum_{i \in I} z_{ik} * V_{i} \leq Fr, \forall k \in T \\
z_{i0} = 0, \forall i \in I \\
x_{ik} + z_{i,k-1} = \sum_{l \in R} y_{lk} * Req_{il} + z_{ik}, \forall k \in T, i \in I \\
\sum_{l \in R} y_{lk} \leq MAX, \forall k \in T \\
y_{lk}, x_{ik}, z_{ik} \geq 0, \forall i \in I, k \in T
\end{align}

- The first constraint ensures there are enough nutrients
- The second constraint ensures that the fridge is not overfilled
- The third constraint sets the intial amount of food to 0
- The fourth constraint ensures that the amount of each ingredient being consumed is balanced by the amount being bought
- The fifth constraint limits the amount of recipes made in a week (i.e. only 7 recipes worth of recipes being made)

In [41]:
# for now leave out multi-period planning and general form - too complex
using Random, NamedArrays

# Data 1:
I = [:rice, :chicken_nuggets, :steak]
N = [:calories, :protein]
R = [:steak_and_rice, :just_nuggets]
Req = NamedArray([5 0 3; 0 10 0], (R, I), ("Recipes", "Ingredients")) 
F = NamedArray([300 0; 200 5; 400 30], (I, N), ("Ingredients", "Nutrients"))
S = NamedArray([2000, 100], N, "Nutrient Requirements")
C = NamedArray([5 5 15; 5 5 15; 5 5 15; 5 5 15; 5 5 15;
    5 5 15; 5 5 15; 5 5 15; 5 5 15; 5 5 15;],
    (T, I), ("Week", "Ingredients"))
T = 1:10
FR_CAP = 1000

#Data 2:
# Random.seed!(300)
# I = ["rice", "chicken nuggets", "steak",
# "apple", "orange", "celery", "yogurt"]
# cost of item is roughly equivalent to string length
# C = [length(I)]
# for i in length(I)
#     C[i] = length(I[i]) + 2 * Random.randn()
# end


# make the model
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

@variable(m, x[I, T] >= 0)
@variable(m, y[R, T] >= 0)

# Indexing Guide:
#
# t - time, i - ingredient, r - recipe, n - nutrient
# x[t, i], y[t, r], Req[r, i], F[i, n]

@constraint(m, nutr_satisfied[n in N, t in T],
    sum(y[r, t] * sum(Req[r, i] * F[i, n] for i in I) for r in R) >= S[n])

@constraint(m, enough_ingr_for_recipes[i in I, t in T],
    x[i, t] >= sum( Req[r, i]*y[r, t] for r in R )) 

@constraint(m, fridge_capacity[t in T], sum( x[i, t] for i in I ) <= FR_CAP)

@objective(m, Min, sum( x[i, t] * C[t, i] for i in I, t in T ))

set_silent(m)
optimize!(m)

ingredient_values = NamedArray( [ (value(x[i, t])) for t in T, i in I ], (T, I), ("Week", "Ingredients"))
recipe_values = NamedArray( [ (value(y[r, t])) for t in T, r in R ], (T, R), ("Week", "Recipes"))

display(ingredient_values)
println()

display(S)
println()

display(F)
println()

println("Recipes cooked per week")
display(recipe_values)
println()

display(Req)
println()

# println("Buy ingredients ", value.(x))
# println("Make recipes ", value.(y))
println("Total cost ", objective_value(m))

10×3 Named Matrix{Float64}
Week ╲ Ingredients │            rice  chicken_nuggets            steak
───────────────────┼──────────────────────────────────────────────────
1                  │         5.55556             -0.0          3.33333
2                  │         5.55556             -0.0          3.33333
3                  │         5.55556             -0.0          3.33333
4                  │         5.55556             -0.0          3.33333
5                  │         5.55556             -0.0          3.33333
6                  │         5.55556             -0.0          3.33333
7                  │         5.55556             -0.0          3.33333
8                  │         5.55556             -0.0          3.33333
9                  │         5.55556             -0.0          3.33333
10                 │         5.55556             -0.0          3.33333

2-element Named Vector{Int64}
Nutrient Requirements  │ 
───────────────────────┼─────
calories               │ 2000
protein                │  100

3×2 Named Matrix{Int64}
Ingredients ╲ Nutrients │ calories   protein
────────────────────────┼───────────────────
rice                    │      300         0
chicken_nuggets         │      200         5
steak                   │      400        30


Recipes cooked per week


10×2 Named Matrix{Float64}
Week ╲ Recipes │ steak_and_rice    just_nuggets
───────────────┼───────────────────────────────
1              │        1.11111             0.0
2              │        1.11111             0.0
3              │        1.11111             0.0
4              │        1.11111             0.0
5              │        1.11111             0.0
6              │        1.11111             0.0
7              │        1.11111             0.0
8              │        1.11111             0.0
9              │        1.11111             0.0
10             │        1.11111             0.0

2×3 Named Matrix{Int64}
Recipes ╲ Ingredients │            rice  chicken_nuggets            steak
──────────────────────┼──────────────────────────────────────────────────
steak_and_rice        │               5                0                3
just_nuggets          │               0               10                0


Total cost 777.7777777777778


In [ ]:
#using DataFrames, CSV
#df = CSV.read("stigler.csv",DataFrame,delim=',')